### Replication of Population-Spatial Analysis “Land Use, Competition, and Migration”

This notebook documents my attempt to ressurect an old, but very interesting paper that was begun by my Master's Thesis advisor and others but came to languish unpublished due to lack of time. My intent going into the project was to take the lead on running a replication of the original analysis that would reflect improvements on several fronts in the years since the original data analysis and tentative finding were produced. If the replication ran as planned, we would have a superior set of models based on better measures that would more clearly demonstrate the basic finding. 

Things did not go as planned. 

This document focuses specifically on some of the more interesting challenges I dealt with in the beginning, middle, and end of the project, and discusses some of the data analysis "life lessons" I gleaned along the way. Should you develop headache, confusion, deja vu, or rage, please discontinue reading.

nb: Articles by Plesser (2018) and Goodman et al. (2006) have greatly influenced the way in which I classify what was being carried out in the three phases of this project: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5778115/
A too-short summary is that:

+ "__Methods reproducibility__: provide sufficient detail about procedures and data so that the same procedures could be exactly repeated.

+ __Results reproducibility__: obtain the same results from an independent study with procedures as closely matched to the original study as possible.

+ __Inferential reproducibility__: draw the same conclusions from either an independent replication of a study or a reanalysis of the original study." (Plesser 2018: para 5)

I present the code generally untouched but with simple markup and some samples of back-and-forth discussions (anonymized) for archival purposes.

The tl:dr for this entire analysis is: replication and inferential reproducibility are hard to achieve, documentation is almost never sufficient to support methods reproducibility, and results that are exciting but not robust can vanish in the wind. 

-Jim




*****************************************

The first coding task was to merge data from a number of different repositories. These data involved individual-level and household-level characteristics measured via survey, community-level characteristics obtained via survey and key informants, and derivative files developed by the Nang Rong Project team at UNC-Chapel Hill. 

This went relatively smoothly. You can almost smell the optimism of a fresh graduate student in my SAS code:

In [ ]:
********************************************************************
**      Program Name: /home/jrhull/paa1996/paamer01.sas
**      Programmer: james r. hull
**      Start Date: 2004 October 21
**      Purpose:
**          Create dataset for Re-write of PAA96 paper:
**          1.) Create multiple datasets based individuals
**              in the 1984 dataset from various existing sources
**          2.) Output datafiles needed to validate merges
**          3.) Merge the multiple datasets into a single dataset
**      Input Data:
**          1.) /nangrong/data_sas/1984/current/indiv84.02
**          2.) /nangrong/data_sas/1994/current/indiv94.03
**          3.) /nangrong/data_sas/1994/current/moved94.02
**          4.) /nangrong/data_sas/1984/current/hh84.01
**          5.) /nangrong/data_sas/1984/current/comm84.01
**          6.) /nangrong/data_sas/sacpc/sacpc068.xpt
**
**      Output Data:
**          1.) /home/jrhull/paa1996/miglu01.xpt
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/nangrong/data_sas/1984/current/indiv84.02';
libname in2 xport '/nangrong/data_sas/1994/current/indiv94.03';
libname in3 xport '/nangrong/data_sas/1994/current/moved94.02';
libname in4 xport '/nangrong/data_sas/1984/current/hh84.01';
libname in5 xport '/nangrong/data_sas/1984/current/comm84.01';
libname in6 xport '/nangrong/data_sas/sacpc/sacpc068.xpt';


libname out1 xport '/home/jrhull/paa1996/miglu01.xpt';

*************************************************************
**  Create template file for use in all subsequent merges  **
*************************************************************;

data template;
   set in1.indiv84;
   keep vill84 house84 cep84;
run;

proc contents data=template position;
run;

***************************************************
**  Create merge1 - using 1984 individuals file  **
***************************************************;

data merge1;
   set in1.indiv84;
   keep vill84 house84 cep84 in84_01 in84_05 in84_06;
   rename in84_01=temp_abs in84_05=sex in84_06=age;
run;

proc contents data=merge1;
run;


***************************************************
**  Create merge2 - using 1994 individuals file  **
***************************************************;

** remove duplicate cases for code 2 migrants**;

data add94in1 drop2a;
   set in2.indiv94;
   keep vill84 house84 cep84 q1 code2;
   rename q1=status;
   if code2=1 | code2=5 then output drop2a;
   else output add94in1;
run;

proc contents data=add94in1 position;
run;

proc contents data=drop2a position;
run;

data add94in2 drop2b;
   set add94in1 (drop=code2);
   if cep84 = . then output drop2b;
   else output add94in2;
run;

proc contents data=add94in2 position;
run;
proc contents data=drop2b position;
run;

proc sort data=add94in2;
   by vill84 house84 cep84;
run;

data merge2 drop2c;
   merge
      template (in=a)
      add94in2 (in=b);
   by vill84 house84 cep84;
   if a=1 then output merge2;
   if a=0 and b=1 then output drop2c;
run;

proc contents data=merge2 position;
run;

proc contents data=drop2c position;
run;

proc freq data=merge2 (keep=status);
run;

******************************************
**  Create merge3 - using moved94 file  **
******************************************;

data add94mov;
   set in3.moved94;
   keep vill84 house84 cep84 mhtype;
run;

proc freq data=add94mov (keep=mhtype);
run;


proc contents data=add94mov position;
run;

data merge3 drop3;
   merge
   template (in=a)
   add94mov (in=b);
   by vill84 house84 cep84;
   if a=1 then output merge3;
   if a=0 and b=1 then output drop3;
run;

proc contents data=merge3 position;
run;

proc contents data=drop3 position;
run;

******************************************
**  Create merge4 - using hh84 file     **
******************************************;

data add84hh;
   set in4.hh84;
   keep vill84 house84 hh84_33 hh84_29 hh84_30 hh84_37-hh84_48;
run;

proc contents data=add84hh position;
run;

data merge4 drop4;
   merge
   template (in=a)
   add84hh (in=b);
   by vill84 house84;
   if a=1 then output merge4;
   if a=0 and b=1 then output drop4;
run;

proc contents data=merge4 position;
run;

proc contents data=drop4 position;
run;

******************************************
**  Create merge5 - using indiv84 file  **
******************************************;

 data add84cnt (keep=vill84 house84 mem3to22);
    set in1.indiv84 (keep=vill84 house84 in84_06);
    by vill84 house84;

    retain mem3to22 0;
    if first.house84 then mem3to22=0;
    if in84_06>2 & in84_06<23 then mem3to22=mem3to22+1;
    if last.house84 then output add84cnt;
run;

proc contents data=add84cnt position;
run;

data add84cnt2 (keep=vill84 house84 memnoind);
    set add84cnt;
    if mem3to22>0 then do;
            memnoind=mem3to22-1;
       end;
       else do;
            memnoind=0;
       end;
   label memnoind='hh members between 3 & 22, minus indiv';
run;

data merge5 drop5;
   merge
   template (in=a)
   add84cnt2 (in=b);
   by vill84 house84;
   if a=1 then output merge5;
   if a=0 and b=1 then output drop5;
run;

proc contents data=merge5 position;
run;
proc contents data=drop5 position;
run;

******************************************
**  Create merge6 - using indiv84 file  **
******************************************;

**  The code below was used in some preliminary
data exploration to determine if numtmpab was correct  **;

/*data test84;
   set in1.indiv84 (keep=vill84 house84 cep84);
run;*/

/* proc freq data=test84 (keep=cep84);
 run;*/

data add84mig (keep=vill84 house84 numtmpab);
    set in1.indiv84 (keep=vill84 house84 cep84);
    by vill84 house84;
    retain numtmpab 0;
    if first.house84 then numtmpab=0;
    if cep84>200 then numtmpab=numtmpab+1;
    if last.house84 then output;
     label numtmpab='Number of Temporarily Absent HH Members';
run;

proc contents data=add84mig position;
run;

proc freq data=add84mig (keep=numtmpab);
run;

data merge6 drop6;
   merge
   template (in=a)
   add84mig (in=b);
   by vill84 house84;
   if a=1 then output merge6;
   if a=0 and b=1 then output drop6;
run;

proc contents data=merge6 position;
run;
proc contents data=drop6 position;
run;

proc freq data=merge6 (keep=numtmpab);
run;



******************************************
**  Create merge7 - using comm84 file  **
******************************************;

data add84com;
   set in5.comm84;
   keep vill84 v84_206;
run;

proc contents data=add84com position;
run;

data merge7 drop7;
   merge
   template (in=a)
   add84com (in=b);
   by vill84;
   if a=1 then output merge7;
   if a=0 and b=1 then output drop7;
run;

proc contents data=merge7 position;
run;

proc contents data=drop7 position;
run;

*******************************************
**  Create merge8 - using sacpc068 file  **
*******************************************;

data addspace;
    set in6.sacpc068;
run;

proc contents data=addspace position;
run;

proc sort data=addspace;
   by vill84;
run;

data merge8 drop8;
    merge
    template (in=a)
    addspace (in=b);
    by vill84;
    if a=1 then output merge8;
    if a=0 and b=1 then output drop8;
run;

proc contents data=merge8 position;
run;

proc contents data=drop8 position;
run;

***********************************************
**  Merge files merge1-merge8 into miglu01   **
***********************************************;

data out1.miglu01;
   merge
   merge1
   merge2
   merge3
   merge4
   merge5
   merge6
   merge7
   merge8;
   by vill84 house84 cep84;
run;

The code above was essentially a brand new attempt to reproduce the findings of the original paper without any reference to the original code, documentation, or modified datasets. I would consider it a "Inferential Replication" more than anything else. We were entirely confident starting out that the major finding was robust and would be more fully revealed with a few simple modifications. 

I worked mostly based off the tables and descriptions in the paper itself, along with a few quick questions to team members who worked on the project along the way, compiling a list of variables to grab or create from the raw underlying data and paying careful attention to things like who was included in the sample, final sample sizes, etc. 

During early meetings with the PIs and original co-authors, the decision was made to make what were deemed to be "minor" changes to the operationalization of the key independent variables, and to drop or add a few controls.

We also opted to take advantage of the latest generation of land-use classification and higher-resolution satellite imagery to re-create the spatial variables for the analysis.

Working on it a few hours a week, it took until the following March to have the brand new dataset pieced together with all the contributions from the other team members (spatial and social).

I ran quite a few checks to compare the finished sample and variable descriptives to the originals (documented in the manuscript).

Here is a taste:

In [ ]:
********************************************************************
**      Program Name: /trainee/jrhull/paa1996/paades01.sas
**      Programmer: james r. hull
**      Start Date: 2005 March 10
**      Purpose:
**        1.) Check the newly created dataset miglu_01 for integrity
**            by comparing it to a random sample of cases from the
**            original datasets that contain its component variables
**      Input Data:
**          1.) /trainee/jrhull/paa1996/miglu01.xpt
**          2.) /nangrong/data_sas/1984/current/hh84.01
**          3.) /nangrong/data_sas/1994/current/indiv94.03
**          4.) /nangrong/data_sas/1984/current/comm84.01
**          5.) /nangrong/data_sas/sacpc/sacpc068.xpt
**          6.) /nangrong/data_sas/1984/current/indiv84.02
**          7.) /nangrong/data_sas/1994/current/moved94.02
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

title1 'Program to check integrity of data in miglu01';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/trainee/jrhull/paa1996/miglu01.xpt';
libname in2 xport '/nangrong/data_sas/1984/current/hh84.01';
libname in3 xport '/nangrong/data_sas/1994/current/indiv94.03';
libname in4 xport '/nangrong/data_sas/1984/current/comm84.01';
libname in5 xport '/nangrong/data_sas/sacpc/sacpc068.xpt';
libname in6 xport '/nangrong/data_sas/1984/current/indiv84.02';
libname in7 xport '/nangrong/data_sas/1994/current/moved94.02';

************************************************************
**  choose a random sample of original cases from hh84    **
**  and cross-check these cases against those in miglu01  **
************************************************************;

data smp5pct1 (keep=vill84 house84 hh84_33 hh84_29 hh84_30 hh84_37-hh84_48);
    set in2.hh84;
    if ranuni(4321) <.05 then output;
run;

data testfile (keep=vill84 house84 hh84_33 hh84_29 hh84_30 hh84_37-hh84_48);
    set in1.miglu01;
run;

proc sort data=testfile out=testfile2 noduprecs;
    by vill84 house84;
run;

data testfile3;
    set testfile2
        smp5pct1;
run;

proc sort data=testfile3 out=testfile4 noduprecs;
    by vill84 house84;
run;


*************************************************************
**  choose a random sample of original cases from indiv94  **
**  and cross-check these cases against those in miglu01   **
*************************************************************;

data smp5pct2 (keep=vill84 house84 cep84 status);
     set in3.indiv94 (keep=vill84 house84 cep84 q1 code2);
     if (code2 ^in(1,5)) and (cep84 ne ' ');
     if ranuni(4321) <.05;
     rename q1=status;
run;

data testfile (keep=vill84 house84 cep84 status);
     set in1.miglu01
     smp5pct2;
run;

proc sort data=testfile out=testfile2 noduprecs;
     by vill84 house84 cep84;
run;

************************************************************
**  choose a random sample of original cases from comm84  **
**  and cross-check these cases against those in miglu01  **
************************************************************;

data smp15pct (keep=vill84 v84_206);
     set in4.comm84 (keep= vill84 v84_206);
     if ranuni (5432) < .15;
run;

data testfile (keep=vill84 v84_206);
     set in1.miglu01;
run;

proc sort data=testfile out=testfile2 noduprecs;
     by vill84;
run;

data testfile3 (keep= vill84 v84_206);
     set testfile2
     smp15pct;
run;

proc sort data=testfile3 out=testfile4 noduprecs;
     by vill84;
run;

**************************************************************
**  choose a random sample of original cases from sacpc068  **
**  and cross-check these cases against those in miglu01    **
**************************************************************;

data smp15pct2;
     set in5.sacpc068;
     if ranuni (5432) < .15;
run;

data testfile (drop= house84 cep84 temp_abs sex age status
     mhtype hh84_29 hh84_30 hh84_33 hh84_37-hh84_48 memnoind
     numtmpab v84_206);
     set in1.miglu01;
run;

proc sort data=testfile out=testfile2 noduprecs;
     by vill84;
run;

data testfile3;
     set testfile2
     smp15pct2;
run;

proc sort data=testfile3 out=testfile4 noduprecs;
     by vill84;
run;

*************************************************************
**  choose a random sample of original cases from indiv84  **
**  and cross-check these cases against those in miglu01   **
*************************************************************;

data smp5pct3;
     set in6.indiv84 (keep=vill84 house84 cep84 in84_06);
     by vill84 house84;

     keep vill84 house84 mem3to22 numtmpab;

     retain mem3to22 numtmpab;

     if first.house84 then do;
                             mem3to22=0;
                             numtmpab=0;
                           end;
     if (3 <= in84_06 <= 22) then mem3to22=mem3to22+1;

     if (cep84 >'200') then numtmpab=numtmpab+1;

     if last.house84 and (ranuni(6543) <.05) then output;
run;

data smp5pct4 (keep=vill84 house84 memnoind numtmpab);
     set smp5pct3;
     if (mem3to22 >0) then memnoind=mem3to22-1;
        else memnoind=0;
run;

data testfile (keep=vill84 house84 memnoind numtmpab);
     set in1.miglu01;
run;

proc sort data=testfile out=testfile2 noduprecs;
     by vill84 house84;
run;

data testfile3;
     set testfile2
     smp5pct4;
run;

proc sort data=testfile3 out=testfile4 noduprecs;
     by vill84 house84;
run;

*****************************************************
**  Still need to think of a way to check moved94  **
*****************************************************;

Things checked out. I next set about fine-tuning the new dataset to resemble the old as closely as possible. At this point, I found myself emailing older graduate students, post-docs, and even graduates or former employees to track down certain vital details about variable operationalization. While it might appear that the variable was adequately described in the tables, there were a number of what Andrew Gelman and others refer to as forks in the "Garden of Forking Paths" that were never documented and (8-10 years on) were completely lost to the memories of the PI and co-authors. But, in nearly every case, a snippet of old code, a footnote, or a recollection of a former analyst began to fill in the blanks. If I recall, we had only 1-2 variables where I truly had to guess at how the originals were transformed from the raw survey responses into the final form for analysis. In those cases, I started with a lengthy conversation with one or more of the lead authors.

In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paades02.sas
**     Programmer: james r. hull
**     Start Date: 2005 March 15
**     Purpose:
**        1.) Create new variables needed to replicate 1996 regression
**            analysis and generate descriptive statistics for all var's
**
**     Input Data:
**        1.) /trainee/jrhull/paa1996/miglu01.xpt
**
**     Output Data:
**        1.) /trainee/jrhull/paa1996/miglu02.xpt
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

title1 'PAA1996: generates variables and descriptive statistics';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/trainee/jrhull/paa1996/miglu01.xpt';
libname out1 xport '/trainee/jrhull/paa1996/miglu02.xpt';

*********************************************************************
**  Run frequencies to see what needs to be recoded for variables  **
*********************************************************************;

/*proc freq data=in1.miglu01;
     title2 "ownership of various assets in 1984";
     tables hh84_29 hh84_30 HH84_38 HH84_40 HH84_42 HH84_44 HH84_46 HH84_48;
run;

proc freq data=in1.miglu01;
     title2 "purpose of ownership questions";
     tables hh84_37 hh84_39 hh84_41 hh84_43 hh84_45 hh84_47;
run;

proc freq data=in1.miglu01;
     title2 "Status variable and mhtype variable";
     tables status mhtype;
run;

proc freq data=in1.miglu01;
     title2 "Electrification";
     tables v84_206;
run;

proc freq data=in1.miglu01;
     title2 "temp_abs";
     tables temp_abs;
run;*/


******************************
**  Create var's: agaset84, agaset_2,
*******************************;

data out1.miglu02 (drop=HH84_29 HH84_30 HH84_33 HH84_37-HH84_48
                   VILLYR REVILLYR TEMP_ABS SEX v84_206);
     set in1.miglu01;

     *** Recode for AGASET84: the 8's and 9's to 0 and . respectively  ***;

     if HH84_29=8 then ITAN=0;
        else if HH84_29=9 then ITAN=.;
        else ITAN=HH84_29;

     if HH84_30=8 then PICKUP=0;
        else PICKUP=HH84_30;

     if HH84_38=98 then CATTLE=0;
        else CATTLE=HH84_38;

     if HH84_40=98 then BUFFALO=0;
        else if HH84_40=99 then BUFFALO=.;
        else BUFFALO=HH84_40;

     if HH84_42=98 then PIGS=0;
        else if HH84_42=99 then PIGS=.;
        else PIGS=HH84_42;

     if HH84_44=98 then GEESE=0;
        else GEESE=HH84_44;

     if HH84_46=98 then DUCKS=0;
        else if HH84_46=99 then DUCKS=.;
        else DUCKS=HH84_46;

     if HH84_48=98 then CHICKENS=0;
        else if HH84_48=99 then CHICKENS=.;
        else CHICKENS=HH84_48;


     *** Create AGASET84 ***;

     *** NOTE: the original variable AGASET84 excluded ITANS
     For this reason, two variables are created, the first
     is exactly the same as the original, the second includes
     the variable measuring ITANS in the total calculation ***;

     AGASET84=(PICKUP*170000) + (CATTLE*8000) + (BUFFALO*6000) +
          (PIGS*2750) + (GEESE*70) + (DUCKS*30) + (CHICKENS*27);

     AGASET_2=(ITAN*170000) + (PICKUP*170000) + (CATTLE*8000) +
          (BUFFALO*6000) + (PIGS*2750) + (GEESE*70) + (DUCKS*30) +
          (CHICKENS*27);


     *** Create VIL_AGE and RVIL_AGE from VILLYR and REVILLYR ***;

     VIL_AGE=(1984-VILLYR);
     RVIL_AGE=(1984-REVILLYR);


     *** CREATE ELECTRIC by re-coding v84_206 2->0 (0 is no)
         1->1 (1 is yes) ***;

     if V84_206=2 then ELECTRIC=0;
     else ELECTRIC=v84_206;


     *** Create MALE by recoding SEX 2->0 (0 is female)
         1->1 (1 is male);

     if SEX=2 then MALE=0;
     else MALE=SEX;


     *** Recode missing values in variable RAI and rename ***;

     if HH84_33=998 then RAI=0;
     else if HH84_33=999 then RAI=.;
     else RAI=HH84_33;


     *** Label newly created variables ***;

     label ITAN='Number of motor/agri cars owned (A3/Q8)'
           PICKUP='Number of pick-up cars owned (A3/Q8)'
           CATTLE='Number of cattle raised (A3/Q12)'
           BUFFALO='Number of buffalo raised (A3/Q12)'
           PIGS='Number of pigs raised (A3/Q12)'
           GEESE='Number of geese raised (A3/Q12)'
           DUCKS='Number of ducks raised (A3/Q12)'
           CHICKENS='Number of chickens raised (A3/Q12)'
           AGASET84='84: Household ag assets'
           AGASET_2='84: Household ag assets (with Itans)'
           VIL_AGE='Village age in 1984'
           RVIL_AGE='Village age in 1984 (logic corrected)'
           ELECTRIC='Electricity in village 0=no 1=yes'
           MALE='Respondent is male 0=female 1=male'
           RAI='How many rai of land HH own? (A3/Q10)';


     *** Select cases in age range desired ***;

     if (3 <= AGE <= 22);

run;

At long last, descriptives time!

In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paades03.sas
**     Programmer: james r. hull
**     Start Date: 2005 March 23
**     Purpose:
**        1.) Create dependent variable needed to replicate 1996 regression
**            analysis and generate descriptive statistics for all var's
**
**     Input Data:
**        1.) /trainee/jrhull/paa/miglu02.xpt
**
**     Output Data:
**        1.) /trainee/jrhull/paa/miglu03.xpt
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: generates variables and descriptive statistics';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/trainee/jrhull/paa1996/miglu02.xpt';
libname out1 xport '/trainee/jrhull/paa1996/miglu03.xpt';

****************************************************************
**  Create Dependent Variable Migration Status - In/Out/LTFU  **
****************************************************************;

*** Frequencies on two questions needed to create DV ***;

/*proc freq data=in1.miglu02;
     table status mhtype;
run;*/

data work1;
     set in1.miglu02;
run;


/*proc freq data=work1;
     tables status mhtype;
run;*/

data work2;
     set work1;
     if STATUS=1 then MIGSTAT=1;
        else if STATUS=2 then MIGSTAT=1;
        else if STATUS=3 then MIGSTAT=2;
        else if STATUS=9 then MIGSTAT=.;
        else if MHTYPE=1 then MIGSTAT=3;
        else if MHTYPE=2 then MIGSTAT=.;
     label MIGSTAT='mig stat 1=in village 2=migrant 3=LTFU';
     if STATUS ^in (0,4);
run;

*** Contents ***;

proc contents data=work2 varnum;
run;

**************************************
**  General descriptive statistics  **
**************************************;

***  remove labels to make less bulky tables and drop ***;
***  cases with missing values on dep. var. migstat   ***;

data work3;
     set work2;
     attrib _all_
         label='';
     if MIGSTAT ne .;
run;

proc freq data=work3;
     tables MIGSTAT /missprint;
run;

proc freq data=work3;
     tables MEMNOIND NUMTMPAB ELECTRIC MALE/missprint;
run;

proc means data=work3 maxdec=2 mean std min max nmiss;
     var AGE MEMNOIND NUMTMPAB G3BPPVIL G3BOLVIL
     G3B: G3D: AGASET84 AGASET_2 VIL_AGE RVIL_AGE RAI;
run;

****************************************
**   Some other descriptive analyses  **
****************************************;

***  Examine the cases originally coded 4 on status  ***;

/*proc sort data=work1 out=work4;
     by status mhtype;
run;

proc freq data=work4;
     by status;
     tables age;
run;*/

***  Generate tables of descriptives by migstat  ***;

proc sort data=work3 out=work5;
     by MIGSTAT;
run;

proc freq data=work5;
     by migstat;
     tables MEMNOIND NUMTMPAB ELECTRIC MALE/missprint;
run;

proc means data=work5 maxdec=2 mean std min max nmiss;
     by migstat;
     var AGE MEMNOIND NUMTMPAB G3BPPVIL G3BOLVIL
     G3B: G3D: AGASET84 AGASET_2 VIL_AGE RVIL_AGE RAI;
run;

data out1.miglu03;
     set work2;
run;

As one note, I won't be sharing any results, even aggregate tables, to avoid any appearance of publishing a work based on the original, highly confidential human subjects data. If you wish to know more about the project, or to obtain IRB approval and access to work with the data, you can start at the U of M ICPSR DSDR site where the data for this project are now maintained: https://www.icpsr.umich.edu/web/DSDR/studies/4402

Eager to get on with the real modeling, I can remember perusing that first batch of descriptives and feeling my heart sink into my stomach. 

Things did not look correct. My first guess was that I had made one (or twenty) mistakes in my code. This was my first big data analysis project using SAS, and mistakes happen. I sent my merge code to a senior programmer on the team with a very polite ask that he review it and give me any pointers (hoping he might also notice any glaring mistakes). 

He sent back some notes I tweaked and re-ran and...things did not look correct.

Not to worry, I counseled myself. If there was a deeper, more hidden mistake in my programming, I had a secret weapon in my toolkit all the while, just waiting for a moment such as this.

While accessing the data and writing all my code over the shared Linux server (since these were highly secure data), I had been able to locate the _original_ datasets that were created almost a decade before and had been carefully archived. Each was numbered, providing a (slightly imperfect) way to match them to each stage of the original analysis. All that was needed was to confirm with the original programmer on the project which version(s) were used to produce which tables. Soon, I had not just one but several versions of the data (there had been an attempt to polish the paper for publication in the intervening years) to compare directly to our own.

Keep in mind that for a subset of the variables, the presumption would be that they would be identical from the original to the c2005 attempt to reproduce. Others, the spatial GIS-derived variables in particular, would be expected to be similar but not identical, owing to things like the improvements in resolution and changes in the classification algorithms. 

So, let's compare these things and get this straightened out...

In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paades04.sas
**     Programmer: james r. hull
**     Start Date: 2005 May 3
**     Purpose:
**        1.) Check out original files used in PAA paper
**        2.) Create file containing all spatial variables
**
**     Input Data:
**        1.)'/nangrong/data_sas/sacpc/sacpc031.xpt'
**        2.)'/nangrong/data_sas/sacpc/sacpc043.xpt'
**        3.)'/nangrong/data_sas/sacpc/sacpc045.xpt'
**        4.)'/nangrong/data_sas/sacpc/sacpc046.xpt'
**        5.)'/nangrong/data_sas/sacpc/sacpc068.xpt'
**        6.)'/nangrong/data_sas/sacpc/sacpc072.xpt'
**
**     Output Data:
**        1.)'/trainee/jrhull/paa1996/spatial.xpt'
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: ';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/nangrong/data_sas/sacpc/sacpc031.xpt';
libname in2 xport '/nangrong/data_sas/sacpc/sacpc043.xpt';
libname in3 xport '/nangrong/data_sas/sacpc/sacpc045.xpt';
libname in4 xport '/nangrong/data_sas/sacpc/sacpc046.xpt';
libname in5 xport '/nangrong/data_sas/sacpc/sacpc068.xpt';
libname in6 xport '/nangrong/data_sas/sacpc/sacpc072.xpt';

libname out1 xport '/trainee/jrhull/paa1996/spatial.xpt';

********************************************************************
**  Bring all datasets into working folder so they can be viewed  **
********************************************************************;


/* proc contents data=work1 varnum;
proc contents data=work2 varnum;
proc contents data=work3 varnum;
proc contents data=work4 varnum;
proc contents data=work5 varnum;
run; */


data work1;
     merge
       in1.sacpc031 (keep=VILL84 GCOP3 GCOB3 GNP73B3 GNP76B3 GFOR73B3 GFOR76B3)
       in2.sacpc043 (keep=VILL84 ORGCOM3K SETCOM3K STCOM3KA)
       in3.sacpc045 (keep=VILL84 G73B3FO G76B3FO)
       in4.sacpc046 (keep=VILL84 GVNP3K73 GVNP3K76)
       in5.sacpc068 (keep=VILL84 G3BPPVIL G3BOLVIL G3B73AF G3D73AF G3B76AF G3D76AF G3B73NF G3B76NF)
       in6.sacpc072;
     by vill84;

     GFOR73B3=GFOR73B3*100;
     GFOR76B3=GFOR76B3*100;

     DIF73FOR=G3B73AF-G73B3FO;
     DIF76FOR=G3B76AF-G76B3FO;
     DIF73NP=G3B73NF-GVNP3K73;
     DIF76NPA=G3B76NF-GVNP3K76;
     DIF76NPB=G3B76NF-GNP76B3;
     DIF76NPC=GVNP3K76-GNP76B3;

run;

proc contents data=work1 varnum;
run;

proc means data=work1 maxdec=2 fw=6 mean stddev min max;
     var GCOP3 ORGCOM3K SETCOM3K STCOM3KA G3BPPVIL;
run;

proc means data=work1 maxdec=2 fw=6 mean stddev min max;
     var GCOB3 G3BOLVIL;
run;

proc means data=work1 maxdec=2 fw=6 mean stddev min max;
     var GNP73B3 GVNP3K73 G3B73NF;
run;

proc means data=work1 maxdec=2 fw=6 mean stddev min max;
     var GNP76B3 GVNP3K76 G3B76NF;
run;

proc means data=work1 maxdec=2 fw=6 mean stddev min max;
     var GFOR73B3 G73B3FO G3B73AF G3D73AF;
run;

proc means data=work1 maxdec=2 fw=6 mean stddev min max;
     var GFOR76B3 G76B3FO G3B76AF G3D76AF;
run;

proc print data=work1;
     id VILL84;
     var GVNP3K73 G3B73NF DIF73NP;
run;

proc print data=work1;
     id VILL84;
     var GNP76B3 GVNP3K76 G3B76NF DIF76NPA DIF76NPB DIF76NPC;
run;

proc print data=work1;
     id VILL84;
     var G73B3FO G3B73AF DIF73FOR;
run;

proc print data=work1;
     id VILL84;
     var G76B3FO G3B76AF DIF76FOR;
run;

proc corr data=work1 noprob nosimple;
     var GNP73B3 GVNP3K73 G3B73NF;
run;

proc corr data=work1 noprob nosimple;
     var GNP76B3 GVNP3K76 G3B76NF;
run;

proc corr data=work1 noprob nosimple;
     var GFOR73B3 G73B3FO G3B73AF;
run;

proc corr data=work1 noprob nosimple;
     var GFOR76B3 G76B3FO G3B76AF;
run;

data out1.spatial;
     set work1;
run;

This deeper dive was a mixed bag. A number of the independent variables and controls were spot on (as one would hope and expect). 

Those variables that were out of line were predominantly the ones we had opted to revise or that were based on updated methodologies.

But, while they looked out-of-line to me, it was difficult for me as a social demographer to really judge how much discrepancy was too much with the spatial variables.

So, after talking with my advisor, I did what nearly any graduate student would do: I ran the models to see if the differences in variable definitions, measurement, and distributions had any appreciable impact on the main results.

In a sense, a crude - very, very crude - form of robustnes analysis. Or, so I told myself at the time.


In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paades05.sas
**     Programmer: james r. hull
**     Start Date: 2005 July 1
**     Purpose:
**        1.) Create dependent variable needed to replicate 1996 regression
**            analysis and generate descriptive statistics for all var's
**        2.) Adds variables in file 'spatial' to final file for testing
**     Input Data:
**        1.) /trainee/jrhull/paa1996/miglu02.xpt
**        2.) /trainee/jrhull/paa1996/spatial.xpt
**     Output Data:
**        1.) /trainee/jrhull/paa/miglu03.xpt
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: generates variables and descriptive statistics';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/trainee/jrhull/paa1996/miglu02.xpt';
libname in2 xport '/trainee/jrhull/paa1996/spatial.xpt';
libname out1 xport '/trainee/jrhull/paa1996/miglu03.xpt';

****************************************************************
**  Create Dependent Variable Migration Status - In/Out/LTFU  **
****************************************************************;

*** Frequencies on two questions needed to create DV ***;

/*proc freq data=in1.miglu02;
     table status mhtype;
run;*/

data work1;
     set in1.miglu02;
run;


/*proc freq data=work1;
     tables status mhtype;
run;*/

data work2;
     merge work1 (in=a)
           in2.spatial (in=b);
     by vill84;

     if STATUS=1 then MIGSTAT=1;
        else if STATUS=2 then MIGSTAT=1;
        else if STATUS=3 then MIGSTAT=2;
        else if STATUS=9 then MIGSTAT=.;
        else if MHTYPE=1 then MIGSTAT=3;
        else if MHTYPE=2 then MIGSTAT=.;
     label MIGSTAT='mig stat 1=in village 2=migrant 3=LTFU';

     if STATUS ^in (0,4);

run;

data out1.miglu03;
     set work2;
run;

In this bit of code, I prepped the SAS dataset for conversion to Stata format, since I was planning to use the mlogit procedure in Stata for the modeling portion of the project. Since the team had worked with SAS almost exclusively for a decade, it seemed natural at the time to do the data wrangling in that language. Plus, I was eager in those days to get practice working with multiple languages and programs.

In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paareg01.sas
**     Programmer: james r. hull
**     Start Date: 2005 April 17
**     Purpose:
**        1.) Performs initial replication of regression in PAA 1996
**            (actually it creates the data sets that will be converted
**            to STATA format so that I can use robust errors in mlogit)
**     Input Data:
**        1.) /trainee/jrhull/paa1996/miglu03.xpt
**
**     Output Data:
**        1.) /trainee/jrhull/paa1996/m03_12_1.xpt
**        2.) /trainee/jrhull/paa1996/m13_22_1.xpt
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: multinomial logistic regression';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/trainee/jrhull/paa1996/miglu03.xpt';
libname out1 xport '/trainee/jrhull/paa1996/m03_12_1.xpt';
libname out2 xport '/trainee/jrhull/paa1996/m13_22_1.xpt';


**********************************************************
**  Create two separate data files based on age groups  **
**********************************************************;

data out1.m03_12_1;
     set in1.miglu03;
     if age < 13;
run;


data out2.m13_22_1;
     set in1.miglu03;
     if age > 12;
run;

Finally, I had some fancy tables to digest. 

I didn't sleep that night. Instead, I pored over the results of the regressions again and again, comparing them to the originals, comparing them to the unpublished first attempted revision that I had also dredged up along the way.

But, nothing matched. There was no sign of a robust, large effect that supported the original hypothesis, which was that young adults in key age ranges would respond to certain visible ecological indications of excessive deforestation in their village by migrating more often to urban areas (where there chances might be better than if they remained and attempted to make a life by farming). Or, flipped around, young adults might be more inclined to stay put if there was still plently of unused forest land in the vicinity that could be converted to pasture or cropland. 

And the tables I was reviewing didn't show that at all. It was like I was looking at two entirely different analyses of two substantially different data sets. Which, as I discovered in what I consider the *second* phase of this project, was entirely correct. I (and others) had woefully underestimated the sensitivity of the original model to what we deemed minor changes while overestimating the robustness of the main (and other) findings.

The very next day, giant comparison table in hand, I trudged over to my Advisor's office to break the news. 

It went suprisingly well, all things considered.

Thus began what I think of as the middle of this project. While we started small, over the next few months we ended up at one point or another roping in everyone who had ever worked on the project to assist us in rooting out the exact source of the differences at every stage of the project. I took the lead and began "stepping" one change at a time from the original model to our latest attempt. In doing so, I began to ammass an incredibly long list of differences, any one of which could be responsible for the ultimate difference in outcomes. 

Some (but certainly not all) of the code from this middle part is included below. This was an incredibly painstaking and time-consuming set of tasks. At each step, before and after the change, a metric had to be produced and compared. In some cases, it might be simple descriptives. In others, I would re-run the model itself. The latter was to help suss out potential interactions between more than one factor that might not be immediately apparent from a simple side-by-side comparison of aggregate statistics. Had I been a better programmer in those days, or more knowledgable about any of this, I might have developed much better, and faster, means of completing this task. 

But, even so, a number of candidate explanations for the "vanishing result" soon emerged.

In [ ]:
********************************************************************
**      Program Name: /trainee/jrhull/paa1996/paafix02.sas
**      Programmer: james r. hull  (With ****s helpful suggestions!)
**      Start Date: 2005 September 11
**      Purpose:
**          1.) Add extra variables to original file for testing
**          2.) This file includes the most recent requests as of 9/11
**
**        Input Data:
**          1.) /nangrong/data_sas/1984/current/indiv84.02
**          2.) /nangrong/data_sas/1994/current/indiv94.03
**          3.) /nangrong/data_sas/1994/current/moved94.02
**          4.) /nangrong/data_sas/1984/current/hh84.01
**          5.) /nangrong/data_sas/1984/current/comm84.01
**          6.) /nangrong/data_sas/sacpc/sacpc068.xpt
**          7.) /trainee/jrhull/paa1996/sacpc035.xpt
**          8.) /trainee/jrhull/paa1996/spatial.xpt
**
**      Output Data:
**          1.) /trainee/jrhull/paa1996/miglu03.xpt
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/nangrong/data_sas/1984/current/indiv84.02';
libname in2 xport '/nangrong/data_sas/1994/current/indiv94.03';
libname in3 xport '/nangrong/data_sas/1994/current/moved94.02';
libname in4 xport '/nangrong/data_sas/1984/current/hh84.01';
libname in5 xport '/nangrong/data_sas/1984/current/comm84.01';
libname in6 xport '/nangrong/data_sas/sacpc/sacpc068.xpt';
libname in7 xport '/trainee/jrhull/paa1996/sacpc035.xpt';

libname in8 xport '/trainee/jrhull/paa1996/spatial.xpt';

libname out1 xport '/trainee/jrhull/paa1996/miglu03.xpt';
libname out2 xport '/trainee/jrhull/paa1996/m03_12_1.xpt';
libname out3 xport '/trainee/jrhull/paa1996/m13_22_1.xpt';
libname out4 xport '/trainee/jrhull/paa1996/m03_12_4.xpt';
libname out5 xport '/trainee/jrhull/paa1996/m13_22_4.xpt';

title1 'Program for testing original model specification';

*---------------------------------------*
*  Get 1994 Individual-Level Variables  *
*---------------------------------------*;

***************************
**  Get Q1 from indiv94  **
***************************;

data ind94;
  set in2.indiv94(keep=VILL84 HOUSE84 CEP84 Q1 CODE2);

** Keep Code 2s in destination and individuals who link to 1984 **;

  if (CODE2 ^in(1,5)) and (CEP84 ne ' ');

** Rename q1 **;

  rename q1=STATUS;

run;


***********************************
**  Concatenate IND94 to MOVE94  **
***********************************;

data indmoved94;
  set ind94(drop=CODE2)
      in3.moved94(keep=VILL84 HOUSE84 CEP84 MHTYPE);

run;

proc sort data=indmoved94 out=sindmoved94 nodupkey;
  by VILL84 HOUSE84 CEP84;
run;


*-----------------------------------------------------*
*  Create and Get 1984 Household-Level Variables      *
*  Create mem3to22 and numtmpab using indiv84 file    *
*-----------------------------------------------------*;

data ind84hh;
  set in1.indiv84(keep=VILL84 HOUSE84 CEP84 IN84_06);
  by VILL84 HOUSE84;

  keep VILL84 HOUSE84 MEM3TO22 NUMTMPAB;

  retain MEM3TO22 NUMTMPAB;

  if first.HOUSE84 then do;
                         MEM3TO22=0;
                         NUMTMPAB=0;
                        end;

  if (3 <= IN84_06 <= 22) then MEM3TO22=MEM3TO22+1;

  if (CEP84 > '200') then NUMTMPAB=NUMTMPAB+1;

  label NUMTMPAB='Number of Temporarily Absent HH Members'
        MEM3TO22='HH Members between 3 & 22';

  if last.HOUSE84 then output;

run;

*----------------------------------------------------*
* Create Village-level number of temp abs HH members *
*----------------------------------------------------*;

data ind84vil;
     set in1.indiv84 (keep=VILL84 HOUSE84 CEP84);
     by VILL84;

     keep VILL84 VILPOP VILTMPAB;

     retain VILTMPAB VILPOP;

     if first.VILL84 then do;
                            VILPOP=0;
                            VILTMPAB=0;
                          end;

     if (CEP84 > '200') then VILTMPAB=VILTMPAB+1;
     if (CEP84 < '200') then VILPOP=VILPOP+1;

     label VILTMPAB='Number Temporarily Absent in Village';
     label VILPOP='Number of non-absent Villagers';

     if last.VILL84 then output;

run;

*************************************************************
**  Merge ind84hh to hh84 to get other hh-level variables  **
*************************************************************;

data hh84 nohh84 noind84hh;
  merge ind84hh(in=a)
        in4.hh84(keep=VILL84 HOUSE84 HH84_33 HH84_29
                      HH84_30 HH84_37-HH84_48 in=b);
  by VILL84 HOUSE84;

  if a=1 and b=1 then output hh84;
  if a=1 and b=0 then output nohh84;
  if a=0 and b=1 then output noind84hh;

run;

*--------------------------------------------------*
*  Get 1984 Community-Level and Spatial Variables  *
*  Merge comm84 to sacpc068                        *
*--------------------------------------------------*;

data commspat84 nospat nocomm84 noind84vil nocpc035;
  merge in5.comm84(keep=VILL84 V84_206 in=a)
        in6.sacpc068(in=b)
        in7.sacpc035(in=c)
        in8.spatial (drop=DIF73FOR DIF73NP DIF76FOR DIF76NPA DIF76NPB DIF76NPC in=d)
        ind84vil (in=e);
  by VILL84;

  TOTPOP=VILTMPAB+VILPOP;
  PCTMIGS=VILTMPAB/TOTPOP*100;

  label TOTPOP='Total Population of Village';
  label PCTMIGS='Percent of Temp. Abs. Villagers';

  if a=1 and b=1 and c=1 and d=1 and e=1 then output commspat84;

run;

*----------------------------------------------------------*
*  Get 1984 Individual-Level Data and Merge in Other Data  *
*  Merge sindmoved94 to indiv84                            *
*----------------------------------------------------------*;

data indiv84 noindmoved94 noindiv84;
  merge in1.indiv84(keep=VILL84 HOUSE84 CEP84 IN84_05 IN84_06 IN84_10
                         rename=(IN84_05=SEX IN84_06=AGE IN84_10=MARSTAT) in=a)
        sindmoved94(in=b);
  by VILL84 HOUSE84 CEP84;

  if a=1 and b=1 then output indiv84;
  if a=1 and b=0 then output noindmoved94;
  if a=0 and b=1 then output noindiv84;

run;

*****************************
**  Merge hh84 to indiv84  **
*****************************;

data indiv84hh nohh84b noindiv84b;
  merge indiv84(in=a)
        hh84(in=b);
  by VILL84 HOUSE84;

  if a=1 and b=1 then output indiv84hh;
  if a=1 and b=0 then output nohh84b;
  if a=0 and b=1 then output noindiv84b;

run;

*************************************
**  Merge commspat84 to indiv84hh  **
*************************************;

data indiv84hhcom nocommspat84 noindiv84hh;
  merge indiv84hh(in=a)
        commspat84(in=b);
  by VILL84;

  if a=1 and b=1 then output indiv84hhcom;
  if a=1 and b=0 then output nocommspat84;
  if a=0 and b=1 then output noindiv84hh;

run;

*----------------------------------------------*
*  Create Variable memnoind from mem3to22      *
*----------------------------------------------*;

data addmemnoi;
  set indiv84hhcom;

*** Create memnoind ***;

if (MEM3TO22 > 0) then MEMNOIND=MEM3TO22-1;
  else MEMNOIND=0;

label MEMNOIND='HH members between 3 & 22, minus indiv';

run;

/* data poptottest;
     set miglu01;
     by VILL84;
     if first.VILL84 then output;
run;

proc print data=poptottest;
     id VILL84;
     var TOTPOPVI TOTPOP VILTMPAB MIGS_REV VILPOP PERSON84 PCMIG84 PCTMIGS;
     sum TOTPOPVI TOTPOP PERSON84;
run; */

/* NEED TO CHANGE FILE NAMES */

data addasset (drop=HH84_29 HH84_30 HH84_33 HH84_37-HH84_48
                   VILLYR REVILLYR SEX v84_206);
     set addmemnoi;

     *** Recode for AGASET84: the 8's and 9's to 0 and . respectively  ***;

     if HH84_29=8 then ITAN=0;
        else if HH84_29=9 then ITAN=.;
        else ITAN=HH84_29;

     if HH84_30=8 then PICKUP=0;
        else PICKUP=HH84_30;

     if HH84_38=98 then CATTLE=0;
        else CATTLE=HH84_38;

     if HH84_40=98 then BUFFALO=0;
        else if HH84_40=99 then BUFFALO=.;
        else BUFFALO=HH84_40;

     if HH84_42=98 then PIGS=0;
        else if HH84_42=99 then PIGS=.;
        else PIGS=HH84_42;

     if HH84_44=98 then GEESE=0;
        else GEESE=HH84_44;

     if HH84_46=98 then DUCKS=0;
        else if HH84_46=99 then DUCKS=.;
        else DUCKS=HH84_46;

     if HH84_48=98 then CHICKENS=0;
        else if HH84_48=99 then CHICKENS=.;
        else CHICKENS=HH84_48;


     *** Create AGASET84 ***;

     *** NOTE: the original variable AGASET84 excluded ITANS;
     *** For this reason, two variables are created, the first;
     *** is exactly the same as the original, the second includes;
     *** the variable measuring ITANS in the total calculation ***;

     AGASET84=(PICKUP*170000) + (CATTLE*8000) + (BUFFALO*6000) +
          (PIGS*2750) + (GEESE*70) + (DUCKS*30) + (CHICKENS*27);

     AGASET_2=(ITAN*170000) + (PICKUP*170000) + (CATTLE*8000) +
          (BUFFALO*6000) + (PIGS*2750) + (GEESE*70) + (DUCKS*30) +
          (CHICKENS*27);


     *** Create VIL_AGE and RVIL_AGE from VILLYR and REVILLYR ***;

     VIL_AGE=(1984-VILLYR);
     RVIL_AGE=(1984-REVILLYR);


     *** CREATE ELECTRIC by re-coding v84_206 2->0 (0 is no)
         1->1 (1 is yes) ***;

     if V84_206=2 then ELECTRIC=0;
     else ELECTRIC=v84_206;


     *** Create MALE by recoding SEX 2->0 (0 is female)
         1->1 (1 is male);

     if SEX=2 then MALE=0;
     else MALE=SEX;


     *** Recode missing values in variable RAI and rename ***;

     if HH84_33=998 then RAI=0;
     else if HH84_33=999 then RAI=.;
     else RAI=HH84_33;


     *** Label newly created variables ***;

     label ITAN='Number of motor/agri cars owned (A3/Q8)'
           PICKUP='Number of pick-up cars owned (A3/Q8)'
           CATTLE='Number of cattle raised (A3/Q12)'
           BUFFALO='Number of buffalo raised (A3/Q12)'
           PIGS='Number of pigs raised (A3/Q12)'
           GEESE='Number of geese raised (A3/Q12)'
           DUCKS='Number of ducks raised (A3/Q12)'
           CHICKENS='Number of chickens raised (A3/Q12)'
           AGASET84='84: Household ag assets'
           AGASET_2='84: Household ag assets (with Itans)'
           VIL_AGE='Village age in 1984'
           RVIL_AGE='Village age in 1984 (logic corrected)'
           ELECTRIC='Electricity in village 0=no 1=yes'
           MALE='Respondent is male 0=female 1=male'
           RAI='How many rai of land HH own? (A3/Q10)';


     *** Select cases in age range desired ***;

     if (3 <= AGE <= 22);

run;

****************************************************************
**  Create Dependent Variable Migration Status - In/Out/LTFU  **
**  Also removes code 0 - dead and code 4 - new, and married  **
****************************************************************;


data adddepvar;
     set addasset (drop=ITAN PICKUP CATTLE BUFFALO PIGS GEESE DUCKS CHICKENS);
     if STATUS=1 then MIGSTAT=1;
        else if STATUS=2 then MIGSTAT=1;
        else if STATUS=3 then MIGSTAT=2;
        else if STATUS=9 then MIGSTAT=.;
        else if MHTYPE=1 then MIGSTAT=3;
        else if MHTYPE=2 then MIGSTAT=.;
     label MIGSTAT='mig stat 1=in village 2=migrant 3=LTFU';

     *** Select cases: unmarried, living in 1994, and in village in 1984 ***;

     if STATUS ^in (0,4)  and MARSTAT ^in (2,9);

     if AGE <13 then AGEGR=1;
        else if AGE > 12 then AGEGR=2;

run;

proc freq data=adddepvar;
     tables MARSTAT STATUS MIGSTAT AGEGR MALE;
run;


proc contents data=adddepvar;
run;

data out1.miglu03;
     set adddepvar;
run;


data adddepvar2;
     set addasset (drop=ITAN PICKUP CATTLE BUFFALO PIGS GEESE DUCKS CHICKENS);
     if STATUS=1 then MIGSTAT=1;
        else if STATUS=2 then MIGSTAT=1;
        else if STATUS=3 then MIGSTAT=2;
        else if STATUS=9 then MIGSTAT=.;
        else if MHTYPE=1 then MIGSTAT=3;
        else if MHTYPE=2 then MIGSTAT=.;
     label MIGSTAT='mig stat 1=in village 2=migrant 3=LTFU';

     *** Select cases: unmarried, living in 1994, and in village in 1984 ***;

     if STATUS ^in (0,4)  and MARSTAT ^in (2,3,4,5,9);

     if AGE <13 then AGEGR=1;
        else if AGE > 12 then AGEGR=2;

run;

proc freq data=adddepvar2;
     tables MARSTAT STATUS MIGSTAT AGEGR MALE;
run;

**********************************************************
**  Create two separate data files based on age groups  **
**********************************************************;

data out2.m03_12_1;
     set adddepvar;
     if age < 13;
run;


data out3.m13_22_1;
     set adddepvar;
     if age > 12;
run;


data out4.m03_12_4;
     set adddepvar2;
     if age < 13;
run;


data out5.m13_22_4;
     set adddepvar2;
     if age > 12;
run;

Above, some possible alterations to the way each variable was operationalized were considered.

Below, an example of swapping the old/new variable lists with the old/new datasets (based on several different measurement approaches).

One thing to note is just how many factors can be different from the old to the new analysis, despite them both being *based on* the same raw dataset.

With so many datasets at different levels and containing really different sorts of things (e.g. a person's age, a village-level gini coefficient, an aggregate statistic summarizing the forest fragmentation in a region around the village), this general task was daunting.


In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paades06.sas
**     Programmer: james r. hull
**     Start Date: 2005 September 30
**     Purpose:
**        1.) Test PAA 1996 model using original data from SACPC044
**        2.) Test " " " using orig. social and new spatial var's
**
**     Input Data:
**
**        1.)'/nangrong/data_sas/sacpc/sacpc031.xpt'
**        2.)'/nangrong/data_sas/sacpc/sacpc043.xpt'
**        3.)'/nangrong/data_sas/sacpc/sacpc045.xpt'
**        4.)'/nangrong/data_sas/sacpc/sacpc046.xpt'
**        5.)'/nangrong/data_sas/sacpc/sacpc068.xpt'
**        6.)'/nangrong/data_sas/sacpc/sacpc072.xpt'
**        7.)'/nangrong/data_sas/paa96/sacpc044.xpt'
**
**     Output Data:
**        1.)'/trainee/jrhull/paa1996/m03_12_2.xpt'
**        2.)'/trainee/jrhull/paa1996/m13_22_2.xpt'
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: ';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/nangrong/data_sas/sacpc/sacpc031.xpt';
libname in2 xport '/nangrong/data_sas/sacpc/sacpc043.xpt';
libname in3 xport '/nangrong/data_sas/sacpc/sacpc045.xpt';
libname in4 xport '/nangrong/data_sas/sacpc/sacpc046.xpt';
libname in5 xport '/nangrong/data_sas/sacpc/sacpc068.xpt';
libname in6 xport '/nangrong/data_sas/sacpc/sacpc072.xpt';
libname in7 xport '/nangrong/data_sas/paa96/sacpc044.xpt';

libname out1 xport '/trainee/jrhull/paa1996/m03_12_2.xpt';
libname out2 xport '/trainee/jrhull/paa1996/m13_22_2.xpt';

*----------------------------------------------------*
*  Compile all spatial variables into a single file  *
*----------------------------------------------------*;

data work1;
     merge
       in1.sacpc031 (keep=VILL84 GCOP3 GCOB3 GNP73B3 GNP76B3 GFOR73B3 GFOR76B3)
       in2.sacpc043 (keep=VILL84 ORGCOM3K SETCOM3K STCOM3KA)
       /* in3.sacpc045 (keep=VILL84 G73B3FO G76B3FO)
       in4.sacpc046 (keep=VILL84 GVNP3K73 GVNP3K76) */
       in5.sacpc068 (keep=VILL84 G3BPPVIL G3BOLVIL G3B73AF G3D73AF G3B76AF G3D76AF G3B73NF G3B76NF)
       in6.sacpc072;
     by vill84;

     GFOR73B3=GFOR73B3*100;   *** Recode from proportion forest to % forest ***;
     GFOR76B3=GFOR76B3*100;

run;

/* proc contents data=in7.sacpc044;
run; */

data work2 nohhleveldata novildata;
     merge in7.sacpc044 (in=a)
           work1 (in=b);
     by vill84;

     if a=1 and b=1 then output work2;
        else if a=1 and b=0 then output novildata;
        else if a=0 and b=1 then output nohhleveldata;
run;

proc contents data=work2;
run;

proc freq data=work2;
     tables agaset84 age84 age84gr elct84 ginicult absent84  mstat84
     hdjute84 pcmig84 person84 q1_1 sex84 where94 where94b;
run;

data work3;
     set work2;

     if sex84=2 then sex84=0;

     if q1_1 ^in (0,9) and where94 ne 4 and mstat84 ne 2 /* and absent84 ne 1 */ ;

run;

proc freq data=work3;
     tables q1_1 where94 where94b age84gr mstat84;
run;

**********************************************************
**  Create two separate data files based on age groups  **
**********************************************************;

data out1.m03_12_2;
     set work3;
     if age84gr =1;
run;


data out2.m13_22_2;
     set work3;
     if age84gr =2;
run;


More of the "swapping" approach. This was a quick and dirty means of turning up promising leads. By effectively holding one set of factors constant between runs and changing a second, the impact could be observed.

If there was no change to the results, that tended to rule out the specific factor altered as a prime suspect. Issues like interaction between multiple changing elements was still a concern, but these were early days. 

In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paafix06.sas
**     Programmer: james r. hull
**     Start Date: 2006 January 04
**     Purpose:
**        1.) Create a file of the matched 1996-2005 data w 2005 variables
**
**     Input Data:
**
**        1.)'/nangrong/data_sas/sacpc/sacpc031.xpt'
**        2.)'/nangrong/data_sas/sacpc/sacpc043.xpt'
**        3.)'/nangrong/data_sas/sacpc/sacpc045.xpt'
**        4.)'/nangrong/data_sas/sacpc/sacpc046.xpt'
**        5.)'/nangrong/data_sas/sacpc/sacpc068.xpt'
**        6.)'/nangrong/data_sas/sacpc/sacpc072.xpt'
**        7.)'/nangrong/data_sas/paa96/sacpc044.xpt'
**
**     Output Data:
**        1.)'/trainee/jrhull/paa1996/m03_12_2.xpt'
**        2.)'/trainee/jrhull/paa1996/m13_22_2.xpt'
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: Creating second version of 1996-2005 matched data file';

**********************
**  Data Libraries  **
**********************;


libname in1 xport '/nangrong/data_sas/1984/current/indiv84.02';
libname in2 xport '/nangrong/data_sas/1994/current/indiv94.03';
libname in3 xport '/nangrong/data_sas/1994/current/moved94.02';
libname in4 xport '/nangrong/data_sas/1984/current/hh84.01';
libname in5 xport '/nangrong/data_sas/1984/current/comm84.01';
libname in6 xport '/nangrong/data_sas/sacpc/sacpc068.xpt';
libname in7 xport '/trainee/jrhull/paa1996/sacpc035.xpt';
libname in8 xport '/trainee/jrhull/paa1996/spatial.xpt';
libname in9 xport '/nangrong/data_sas/paa96/sacpc044.xpt';

libname out1 xport '/trainee/jrhull/paa1996/m03_12_5.xpt';
libname out2 xport '/trainee/jrhull/paa1996/m13_22_5.xpt';




*---------------------------------------*
*  Get 1994 Individual-Level Variables  *
*---------------------------------------*;

***************************
**  Get Q1 from indiv94  **
***************************;

data ind94;
  set in2.indiv94(keep=VILL84 HOUSE84 CEP84 Q1 CODE2);

** Keep Code 2s in destination and individuals who link to 1984 **;

  if (CODE2 ^in(1,5)) and (CEP84 ne ' ');

** Rename q1 **;

  rename q1=STATUS;

run;


***********************************
**  Concatenate IND94 to MOVE94  **
***********************************;

data indmoved94;
  set ind94(drop=CODE2)
      in3.moved94(keep=VILL84 HOUSE84 CEP84 MHTYPE);

run;

proc sort data=indmoved94 out=sindmoved94 nodupkey;
  by VILL84 HOUSE84 CEP84;
run;


*-----------------------------------------------------*
*  Create and Get 1984 Household-Level Variables      *
*  Create mem3to22 and numtmpab using indiv84 file    *
*-----------------------------------------------------*;

data ind84hh;
  set in1.indiv84(keep=VILL84 HOUSE84 CEP84 IN84_06);
  by VILL84 HOUSE84;

  keep VILL84 HOUSE84 MEM3TO22 NUMTMPAB;

  retain MEM3TO22 NUMTMPAB;

  if first.HOUSE84 then do;
                         MEM3TO22=0;
                         NUMTMPAB=0;
                        end;

  if (3 <= IN84_06 <= 22) then MEM3TO22=MEM3TO22+1;

  if (CEP84 > '200') then NUMTMPAB=NUMTMPAB+1;

  label NUMTMPAB='Number of Temporarily Absent HH Members'
        MEM3TO22='HH Members between 3 & 22';

  if last.HOUSE84 then output;

run;

*----------------------------------------------------*
* Create Village-level number of temp abs HH members *
*----------------------------------------------------*;

data ind84vil;
     set in1.indiv84 (keep=VILL84 HOUSE84 CEP84);
     by VILL84;

     keep VILL84 VILPOP VILTMPAB;

     retain VILTMPAB VILPOP;

     if first.VILL84 then do;
                            VILPOP=0;
                            VILTMPAB=0;
                          end;

     if (CEP84 > '200') then VILTMPAB=VILTMPAB+1;
     if (CEP84 < '200') then VILPOP=VILPOP+1;

     label VILTMPAB='Number Temporarily Absent in Village';
     label VILPOP='Number of non-absent Villagers';

     if last.VILL84 then output;

run;

*************************************************************
**  Merge ind84hh to hh84 to get other hh-level variables  **
*************************************************************;

data hh84 nohh84 noind84hh;
  merge ind84hh(in=a)
        in4.hh84(keep=VILL84 HOUSE84 HH84_33 HH84_29
                      HH84_30 HH84_37-HH84_48 in=b);
  by VILL84 HOUSE84;

  if a=1 and b=1 then output hh84;
  if a=1 and b=0 then output nohh84;
  if a=0 and b=1 then output noind84hh;

run;

*--------------------------------------------------*
*  Get 1984 Community-Level and Spatial Variables  *
*  Merge comm84 to sacpc068                        *
*--------------------------------------------------*;

data commspat84 nospat nocomm84 noind84vil nocpc035;
  merge in5.comm84(keep=VILL84 V84_206 in=a)
        in6.sacpc068(in=b)
        in7.sacpc035(in=c)
        in8.spatial (drop=DIF73FOR DIF73NP DIF76FOR DIF76NPA DIF76NPB DIF76NPC in=d)
        ind84vil (in=e);
  by VILL84;

  TOTPOP=VILTMPAB+VILPOP;
  PCTMIGS=VILTMPAB/TOTPOP*100;

  label TOTPOP='Total Population of Village';
  label PCTMIGS='Percent of Temp. Abs. Villagers';

  if a=1 and b=1 and c=1 and d=1 and e=1 then output commspat84;

run;

*----------------------------------------------------------*
*  Get 1984 Individual-Level Data and Merge in Other Data  *
*  Merge sindmoved94 to indiv84                            *
*----------------------------------------------------------*;

data indiv84 noindmoved94 noindiv84;
  merge in1.indiv84(keep=VILL84 HOUSE84 CEP84 IN84_05 IN84_06 IN84_10
                         rename=(IN84_05=SEX IN84_06=AGE IN84_10=MARSTAT) in=a)
        sindmoved94(in=b);
  by VILL84 HOUSE84 CEP84;

  if a=1 and b=1 then output indiv84;
  if a=1 and b=0 then output noindmoved94;
  if a=0 and b=1 then output noindiv84;

run;

*****************************
**  Merge hh84 to indiv84  **
*****************************;

data indiv84hh nohh84b noindiv84b;
  merge indiv84(in=a)
        hh84(in=b);
  by VILL84 HOUSE84;

  if a=1 and b=1 then output indiv84hh;
  if a=1 and b=0 then output nohh84b;
  if a=0 and b=1 then output noindiv84b;

run;

*************************************
**  Merge commspat84 to indiv84hh  **
*************************************;

data indiv84hhcom nocommspat84 noindiv84hh;
  merge indiv84hh(in=a)
        commspat84(in=b);
  by VILL84;

  if a=1 and b=1 then output indiv84hhcom;
  if a=1 and b=0 then output nocommspat84;
  if a=0 and b=1 then output noindiv84hh;

run;

*----------------------------------------------*
*  Create Variable memnoind from mem3to22      *
*----------------------------------------------*;

data addmemnoi;
  set indiv84hhcom;

*** Create memnoind ***;

if (MEM3TO22 > 0) then MEMNOIND=MEM3TO22-1;
  else MEMNOIND=0;

label MEMNOIND='HH members between 3 & 22, minus indiv';

run;

/* data poptottest;
     set miglu01;
     by VILL84;
     if first.VILL84 then output;
run;

proc print data=poptottest;
     id VILL84;
     var TOTPOPVI TOTPOP VILTMPAB MIGS_REV VILPOP PERSON84 PCMIG84 PCTMIGS;
     sum TOTPOPVI TOTPOP PERSON84;
run; */

/* NEED TO CHANGE FILE NAMES */

data addasset (drop=HH84_29 HH84_30 HH84_33 HH84_37-HH84_48
                   VILLYR REVILLYR SEX v84_206);
     set addmemnoi;

     *** Recode for AGASET84: the 8's and 9's to 0 and . respectively  ***;

     if HH84_29=8 then ITAN=0;
        else if HH84_29=9 then ITAN=.;
        else ITAN=HH84_29;

     if HH84_30=8 then PICKUP=0;
        else PICKUP=HH84_30;

     if HH84_38=98 then CATTLE=0;
        else CATTLE=HH84_38;

     if HH84_40=98 then BUFFALO=0;
        else if HH84_40=99 then BUFFALO=.;
        else BUFFALO=HH84_40;

     if HH84_42=98 then PIGS=0;
        else if HH84_42=99 then PIGS=.;
        else PIGS=HH84_42;

     if HH84_44=98 then GEESE=0;
        else GEESE=HH84_44;

     if HH84_46=98 then DUCKS=0;
        else if HH84_46=99 then DUCKS=.;
        else DUCKS=HH84_46;

     if HH84_48=98 then CHICKENS=0;
        else if HH84_48=99 then CHICKENS=.;
        else CHICKENS=HH84_48;


     *** Create AGASET84 ***;

     *** NOTE: the original variable AGASET84 excluded ITANS
     For this reason, two variables are created, the first
     is exactly the same as the original, the second includes
     the variable measuring ITANS in the total calculation ***;

     AGASET84=(PICKUP*170000) + (CATTLE*8000) + (BUFFALO*6000) +
          (PIGS*2750) + (GEESE*70) + (DUCKS*30) + (CHICKENS*27);

     AGASET_2=(ITAN*170000) + (PICKUP*170000) + (CATTLE*8000) +
          (BUFFALO*6000) + (PIGS*2750) + (GEESE*70) + (DUCKS*30) +
          (CHICKENS*27);


     *** Create VIL_AGE and RVIL_AGE from VILLYR and REVILLYR ***;

     VIL_AGE=(1984-VILLYR);
     RVIL_AGE=(1984-REVILLYR);


     *** CREATE ELECTRIC by re-coding v84_206 2->0 (0 is no)
         1->1 (1 is yes) ***;

     if V84_206=2 then ELECTRIC=0;
     else ELECTRIC=v84_206;


     *** Create MALE by recoding SEX 2->0 (0 is female)
         1->1 (1 is male);

     if SEX=2 then MALE=0;
     else MALE=SEX;


     *** Recode missing values in variable RAI and rename ***;

     if HH84_33=998 then RAI=0;
     else if HH84_33=999 then RAI=.;
     else RAI=HH84_33;


     *** Label newly created variables ***;

     label ITAN='Number of motor/agri cars owned (A3/Q8)'
           PICKUP='Number of pick-up cars owned (A3/Q8)'
           CATTLE='Number of cattle raised (A3/Q12)'
           BUFFALO='Number of buffalo raised (A3/Q12)'
           PIGS='Number of pigs raised (A3/Q12)'
           GEESE='Number of geese raised (A3/Q12)'
           DUCKS='Number of ducks raised (A3/Q12)'
           CHICKENS='Number of chickens raised (A3/Q12)'
           AGASET84='84: Household ag assets'
           AGASET_2='84: Household ag assets (with Itans)'
           VIL_AGE='Village age in 1984'
           RVIL_AGE='Village age in 1984 (logic corrected)'
           ELECTRIC='Electricity in village 0=no 1=yes'
           MALE='Respondent is male 0=female 1=male'
           RAI='How many rai of land HH own? (A3/Q10)';


     *** Select cases in age range desired ***;

     if (3 <= AGE <= 22);

run;

****************************************************************
**  Create Dependent Variable Migration Status - In/Out/LTFU  **
**  Also removes code 0 - dead and code 4 - new, and married  **
****************************************************************;


data adddepvar;
     set addasset (drop=ITAN PICKUP CATTLE BUFFALO PIGS GEESE DUCKS CHICKENS);
     if STATUS=1 then MIGSTAT=1;
        else if STATUS=2 then MIGSTAT=1;
        else if STATUS=3 then MIGSTAT=2;
        else if STATUS=9 then MIGSTAT=.;
        else if MHTYPE=1 then MIGSTAT=3;
        else if MHTYPE=2 then MIGSTAT=.;
     label MIGSTAT='mig stat 1=in village 2=migrant 3=LTFU';

     if AGE <13 then AGEGR=1;
        else if AGE > 12 then AGEGR=2;

run;

proc freq data=adddepvar;
     tables MARSTAT STATUS MIGSTAT AGEGR MALE;
run;

*------------------------------------------------------*
* Match 1996 cases to 2005 cases, using 2005 variables *
*------------------------------------------------------*;

data matchtemp no05data no96data;
      merge in9.sacpc044 (in=a)
           adddepvar (in=b);
     by vill84 house84 cep84;

     if a=1 and b=1 then output matchtemp;
     if a=1 and b=0 then output no05data;
     if a=0 and b=1 then do;
                            LOSTCASE=1;
                            output no96data;
                         end;
run;

proc sort data=matchtemp out=matchtemp2 nodupkey;
     by vill84 house84 cep84;
run;

proc contents data=matchtemp;
run;

proc freq data=matchtemp;
     tables migstat male agegr /* lostcase */ ;
run;

data match9605;
     set matchtemp;

     *** Select cases that are found in both 1996 file and 2005 file ***;

     /* if LOSTCASE ne 1;  */

     *** Select cases: unmarried, living in 1994, and in village in 1984 ***;

     if STATUS ^in (0,4)  and MARSTAT ^in (2,3,4,5,9);

run;

**********************************************************
**  Create two separate data files based on age groups  **
**********************************************************;

/* SEE DISTINCTIONS ABOVE */

data out1.m03_12_5;
     set match9605;
     if agegr=1;
run;

data out2.m13_22_5;
     set match9605;
     if agegr=2;
run;


Having spent quite some time in producing parallel versions of the dataset (ranging from the original by steps to our naive first attempt in 2005), as well as complete lists of all variables used at any point, matched to each reproduction, I now set about a more sytematic approach.

In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/paa1996/paafix06.sas
**     Programmer: james r. hull
**     Start Date: 2006 January 04
**     Purpose:
**        1.) Generate contents lists for all current data files
**
**     Input Data:
**        1.) '/trainee/jrhull/paa1996/m03_12_1.xpt';
**        2.) '/trainee/jrhull/paa1996/m13_22_1.xpt';
**        3.) '/trainee/jrhull/paa1996/m03_12_2.xpt';
**        4.) '/trainee/jrhull/paa1996/m13_22_2.xpt';
**        5.) '/trainee/jrhull/paa1996/m03_12_3.xpt';
**        6.) '/trainee/jrhull/paa1996/m13_22_3.xpt';
**        7.) '/trainee/jrhull/paa1996/m03_12_4.xpt';
**        8.) '/trainee/jrhull/paa1996/m13_22_4.xpt';
**        9.) '/trainee/jrhull/paa1996/m03_12_5.xpt';
**        10.) '/trainee/jrhull/paa1996/m13_22_5.xpt';
**
**
**     Output Data:
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=55;

title1 'PAA1996: Contents for all current data files';

**********************
**  Data Libraries  **
**********************;

libname in1 xport '/trainee/jrhull/paa1996/m03_12_1.xpt';
libname in2 xport '/trainee/jrhull/paa1996/m13_22_1.xpt';
libname in3 xport '/trainee/jrhull/paa1996/m03_12_2.xpt';
libname in4 xport '/trainee/jrhull/paa1996/m13_22_2.xpt';
libname in5 xport '/trainee/jrhull/paa1996/m03_12_3.xpt';
libname in6 xport '/trainee/jrhull/paa1996/m13_22_3.xpt';
libname in7 xport '/trainee/jrhull/paa1996/m03_12_4.xpt';
libname in8 xport '/trainee/jrhull/paa1996/m13_22_4.xpt';
libname in9 xport '/trainee/jrhull/paa1996/m03_12_5.xpt';
libname in10 xport '/trainee/jrhull/paa1996/m13_22_5.xpt';

proc contents data=in1.m03_12_1;
run;

proc contents data=in3.m03_12_2;
run;

proc contents data=in5.m03_12_3;
run;

proc contents data=in7.m03_12_4;
run;

proc contents data=in9.m03_12_5;
run;


The goal at this point had become to simply repeat or recreate exactly both the original and first attempted revision. 

In many ways, this was the most educational and surprising portion of the project for me, as it was only at this stage that I came to fully appreciate just how many absolutely unremarkable, tiny, and (so was thought) inconsequential decisions were made along the way.

One reason this stage took a long time was that we lacked one critical thing from those earlier attempts: the *code* itself. We had the actual datasets (from which much, though not all, could be inferred backwards), the finished tables, and a good deal of supplemental material (emails, conversations, memos), but not the exact code used for all those earlier versions and analyses. 

Giant life lesson #1: Keep *all* code you ever write and use. If you publish and are able to, put that code out somewhere in the world for others to review/build on.

But, I soldiered on, creeping ever-closer to the *exact* series of steps taken. These included things like:
+ variable definitions
+ coding decisions
+ missing data considerations
+ grouping and categorization rules
+ thresholds and cutoffs for transforming continuous variables into categories, thresholds and boundaries for spatial calculations
+ sample inclusion and exclusion criteria (i.e. should I consider X a member of population Y or Y'?)
+ modeling considerations (e.g. modeling clustering explicitly versus treating it as form of bias to remove)
+ legitimate fixes to the data that had occurred between rounds (meaning the old analyses may have been based in part n data that was flawed)
+ cleaning or other "improvements" to the data that occurred between rounds
+ information obtained during a later round of longitudinal collection that invalidates what was a reasonable assumption at an earlier point in time (and hence can no longer be maintained)
+ uncovering mistakes and errors in the *original* analysis that were automatically corrected in the later ones without realizing it 
+ differences in the precise timing of the images that became the basis for the land use classification (wet/dry season effects)
+ resolution, pixel-size, and re-sampling of sattelite data used in classifications (and other factors) - at one point I made a list of roughly 220 variations on the set of nine spatial forest metrics we had as options 
+ number of classes used in classification
+ method of classification (unsupervised, supervised, hybrid)
+ silly, silly errors in our own attempt to reproduce the original analysis (we found at least one on careful review, that resulted from a fascinating mis-communication across disciplinary lines - this story also involved the words "donut hole" and "fragstats" fwiw


Some of these may seem customary, while others might be a bit surprising. But, with a large, complex and ongoing (longitudinal) data collection effort, the assumption that the data are a static entity, carefully encased in cellophane wrapping, is not supportable. 

Here's an example of a single item from a very long list I presented to our PIs and co-authors in late 2005:

"3.)	SACPC031 village points are based upon the 12/95 version of the 1984 topographic base maps – file vill9512: n=809
SACPC043,045, & 046 village points are based upon **** *****’s Spring 1997 field work – file vc9712: n=879
SACPC068 village points are based upon original 1984 topographic base maps – file vill9512: n=809, but in conversation with Phil, I learned that in constructing these new variables, he used only the subset of the 51 “study villages” to do the calculations, this is the explanation for the large differences between previous analyses and the most recent replication."

What does it mean? In short, our understanding of the social and geographic history of the villages in our dataset was an evolving product. With each wave of data collection, as well as with regular return visits, conversations with colleagues on the ground there, and independent research projects like the Field work of a graduate student, our understanding changed, mostly for the better. Collectively, the base of knowledge about the underlying subject matter was being broadened and deepened, but as a consequence, some of the things that were done in earlier times seemed quaint or downright wrong in hindsight.

Things like hindsight as sources of variation in results are tricky to diagnose because of the hard work that is done instilling in graduate students the need to revise, update, and challenge assumptions. The current paradigm is just part of the water we swim in, so it is extremely useful when faced with this sort of situation to arrange for "inter-generational" conversations between folks who were present at different stages or between senior and junior researchers. It was only through these imperfect but invaluable face-to-face conversations that some of these changes were surfaced. Had we not convened everyone together from the early and the later days, these changes would likely have remained buried as notes in long-dead email threads.

What follows is far less interesting - a series of slightly different Stata do-files, as I zeroed in on the original model:


In [ ]:
/* NOTE: This file used only to test models using 1996 and 1998 spatial variables */
/* It looks very much like the 'normal' do-file, and could easily be confused */

/*****************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering */
/*****************************************************************************************/

log close
log using paafix02, replace text

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, robust cluster (vill84) 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



/*****************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1998 paper, w/ clustering */
/*****************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo, robust cluster (vill84) 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo, basecategory(1) robust cluster (vill84) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo, basecategory(1) robust cluster (vill84)




/*****************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1998 paper, w/ clustering */
/*****************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, robust cluster (vill84) 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1) robust cluster (vill84) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) robust cluster (vill84)


/********************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996/8 paper, w/o clustering */
/********************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3
           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1)  
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1)  



/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo
           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo

/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo, basecategory(1)  
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo, basecategory(1) 


/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1)  
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) 



/**********************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996/8 paper, w/ HH clustering */
/**********************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, robust cluster (house84) 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, robust cluster (house84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (house84) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (house84) 



/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo, robust cluster (house84) 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo, robust cluster (house84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo, basecategory(1) robust cluster (house84) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo, basecategory(1) robust cluster (house84)



/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, robust cluster (house84) 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, robust cluster (house84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1) robust cluster (house84) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) robust cluster (house84)




More of the same:

In [ ]:
/* NOTE: This file used only to test models using 1996 and 1998 spatial variables */
/* USING THE SVYMLOGIT COMMAND */
/* It looks very much like the 'normal' do-file, and could easily be confused */

/*****************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering */
/*****************************************************************************************/

log close
log using paafix03, replace text

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3 

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1)  
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1)  



/*****************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1998 paper, w/ clustering */
/*****************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

/* Regression: 3-12, attept to replicate original 1998 analysis exactly w/cluster*/

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo

           
/*1976*/

/* Regression: 3-12, attempt to replicate original 1998 analysis exactly w/cluster */

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster*/

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k73 g73b3fo, basecategory(1) 
 
/*1976*/

/* Regression: 13-22, attempt to replicate original 1998 analysis exactly w/cluster */

svyset, psu(vill84)
svy:mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka gvnp3k76 g76b3fo, basecategory(1)







And more Stata code:

In [ ]:
/* NOTE: This file used only to test models using spatial var's based on recreated 1996 and 2005 classifications */
/* It looks very much like many other do-files created so far, and could easily be confused */

log close
log using paafix04, replace text

/*****************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering             */
/* THESE MODELS USE NEWLY CREATED VERSIONS OF THE SPATIAL VARIABLES BASED ON THE 2005 CLASSIFICATION */
/*****************************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n73f05 g3p73f05, robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n76f05 g3p76f05, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n73f05 g3p73f05, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n76f05 g3p76f05, basecategory(1) robust cluster (vill84) 


/*****************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering             */
/* THESE MODELS USE NEWLY CREATED VERSIONS OF THE SPATIAL VARIABLES BASED ON THE 1996 CLASSIFICATION */
/*****************************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n73f96 g3p73f96, robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n76f96 g3p76f96, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

/* Regression: 13-22 */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n73f96 g3p73f96, basecategory(1) robust cluster (vill84) 
 
/*1976*/

/* Regression: 13-22 */
mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 gcob3 g3n76f96 g3p76f96, basecategory(1) robust cluster (vill84) 


/*****************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1998 paper, w/ clustering             */
/* THESE MODELS USE NEWLY CREATED VERSIONS OF THE SPATIAL VARIABLES BASED ON THE 2005 CLASSIFICATION */
/*****************************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n73f05 g3p73f05, robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n76f05 g3p76f05, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n73f05 g3p73f05, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n76f05 g3p76f05, basecategory(1) robust cluster (vill84)


/*****************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1998 paper, w/ clustering             */
/* THESE MODELS USE NEWLY CREATED VERSIONS OF THE SPATIAL VARIABLES BASED ON THE 1996 CLASSIFICATION */
/*****************************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta

/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n73f96 g3p73f96, robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n76f96 g3p76f96, robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n73f96 g3p73f96, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit migstat male age pcmig84 elct84 person84 ginicult hdjute84 stcom3ka g3n76f96 g3p76f96, basecategory(1) robust cluster (vill84)



And yet more Stata Code:

In [ ]:
/* NOTE: This file used to test models using the EXACT ORIGINAL DATA from SACPC044 w/ old & new spatial var's */
/* As usual, it looks very much like many other do-files created so far, and could easily be confused         */

log close
log using paafix06, replace text

/********************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering - BUF Comp Var */
/********************************************************************************************************/

/* AGES 3-12 */

use m03_12_2.dta

/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_2.dta


/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/*********************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering  - PIP Comp Var */
/*********************************************************************************************************/

/* AGES 3-12 */

use m03_12_2.dta

/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcop3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcop3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_2.dta


/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcop3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcop3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



OK, I could keep on with those (there were many iterations), but clearly you have the gist. I ran and re-ran a lot of models, descriptives, and other metrics for comparison.

While I was busy with this over several months, the geographers on our team were hard at work on a similar process of comparison and differencing across the many iterations of their spatial classification models.

As an interesting, if infuriating, note. At the time, the social scientists on the team were somewhat aghast to learn that, owing to certain stochastic features of some of the classification algorithms, it was not possible in practice to repeat or re-create the exact results that would have been used in the original version of the paper.

That these intermediate steps were lost to history was not inevitable, but all-too-common. This was more than just a matter of setting a random seed, though the specifics of the process were a bit outside my ouvre then (and certainly remain so today). But as a result, the hope that we could fully repeat that aspect of the analysis was dashed. Even the software worked against it with improvements and changes to code and to algorithms. To carry it out (in principle) would have required that we build authentic hardware and software replicas from 10 years prior (or really good emulators) and re-run the pertinent portions of the analysis along with parameters that were never recorded or preserved.

It was all quite maddening, but in the most interesting ways.

By the by, we narrowed down our long list of possible candidates for the "big factor" that was responsible for our disappearing finding to the spatial data. While there were initially many small discrepancies between the original and the new versions of the survey-based social data (on individuals, households, and communities), these were systematically removed and then re-added one-by-one with little impact.

With a few final versions of the model, I was able to bring to the team a fairly strong set of "results about the results".

A snippet from a contemporaneous memo I drafted describes my hunch:

"The village competition variables show little change.  It is the pattern metric and composition variables that have changed dramatically.  Following the means and other descriptives are simple correlation tables for the same groups of variables.  There is actually a negative correlation between the old and new values for number of forest patches for both years.  Lastly, I include a set of scatterplots that contrast the 2005 variables with the more recent 1998 variables.  From these, the lack of correspondence between the patch count variables is visually apparent.  The composition variables (% forest) show a pattern that one would anticipate, with the exception that nearly all villages show an increase or no change in % forest cover.

The differences in these variables are substantial enough that they change the magnitude, significance, and in some cases, direction of the coefficients in the models.  My best hypothesis is that these differences are the result of the re-classification of the images that took place between 1998 and 2005.  My principal question is should I invest my time in attempting to reconcile these differences, or would we as a group prefer to use the earlier variable versions, based on older classification schemes?"

The question at the end was especially fraught for a graduate student. In effect: how much more time would you have me pour into this black hole? On the other hand, would it even be defensible or right to walk all the back to the decades-old analysis and attempt to move ahead with those data and analysis (effectively dumping the past year's work into the wastebasket)? The obvious answer, given the few glaring mistakes and other concerns I had already uncovered with the original analysis was that it would be a difficult case to argue that it was the superious model (especially with the new analysis staring us in the face). I knew that my vote, at least, was to never put my name on that sort of thing. I also knew that, in all likelihood, if we did try to revert back to the older analysis, my contribution to the whole effort might be so diminished as to bump me to the last author, or right off the masthead altogether and into a footnote. 

In [ ]:
/* NOTE: This file used to test models using the EXACT ORIGINAL DATA AND EXACT VARIABLES from SACPC044 w/ old & new spatial var's */
/* As usual, it looks very much like many other do-files created so far, and could easily be confused         */

log close
log using paafix08, replace text

/********************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering - BUF Comp Var */
/* DATA ARE THE ORINGINAL 17,342 CASES FROM THE FILE SACPC044, MINUS EXCLUSIONS                         */
/********************************************************************************************************/

/* AGES 3-12 */

use m03_12_2.dta

/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_2.dta


/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) robust cluster (vill84) 



/*********************************************************************************************************/
/* Regression of approximately all ORIGINAL variables from the 1996 paper, w/ clustering - BUF Comp Var  */
/* DATA ARE THE 17,281 CASES THAT MATCHED BETWEEN SACPC044 & CURRENT DATA, MINUS EXCLUSIONS (53, NOT 61) */
/*********************************************************************************************************/

/* AGES 3-12 */

use m03_12_3.dta

/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) robust cluster (vill84) 



/* AGES 13-22 */

use m13_22_3.dta


/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k73 g73b3fo, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 setcom3k gvnp3k76 g76b3fo, basecategory(1) robust cluster (vill84) 




A big coming-together was scheduled. In preparation, I was able to successfully summarize all the things we had learned in the course of what amounted to a large-scale sensitivity analysis in a series of tables and visuals. These contained:
+ 5 different sets of independent variables and controls (changes to the model)
+ 2 approaches to measuring village competition (point-in-polygon vs. overlapping buffers)
+ 5 different sets of underyling data (each containing every specification of every variable)
+ 2 different classification years
+ 2 different age groups

In total this was 200 specifically-chosen models (out of hundreds of thousands).

Before moving to the final part of the project, it's worth reflecting on whether this whole exercise transformed into a data-dredging exercise at some point. To be honest, it was something I began to worry about early in the process. Perhaps that's one reason why, as a group, we resisted the principle temptation in this whole affair, which would have been to simply select whichever of the 200+ models had the best appearance and characteristics (pick you metrics) and hasten that one to the publishers with a suitable set of post-hoc explanations.

This, to be clear, was never our modus operandi. Instead, and thankfully, everyone on board was invested in trying to understand *what* had happened, and if possibly *why* or *how* it had happened. Most of all, we wanted to understand what its implications were, in the immediate term for the survivability of the original paper, but in the longer term for our understanding of the complex and coupled human-environmental system that we were studying. 

Put differently, we were engaged in asking a long series of "what if" questions with the goal not of improving model fit or some such thing, but with the tandem goals of a.) mapping all possible differences between the original highly complex analysis and the contemporary highly complex analysis. While I didn't get into it except to point out the possibility earlier, there were also strong indications that some of the most important "effects" observed in the original paper were likely to have been reproducible only through the interaction of more than one factor, at least one of which was, technically speaking, impossible to reproduce 10 years later. 

Whatever else had happened during the original analysis (done in far less time as a rough pass in support of a conference presentation), one could reasonably infer that the team just got luckly. A cynic might go on to infer that something sinister occurred when the first analysis was conducted, amounting to the kind of specification search alluded to above. But, having sifted through veritable file drawers of alternative results and experienced firsthand the dizzying complexity of the problem, I am left to infer only this: good instincts and bad luck are the most likely culprits. The team, free to exercise some judgment in moving from theory to measurement and modeling, made some excellent choices (not realizing what might have obtained in all the other scenarios). This is not uncommon in the traditional confirmatory research tradition, where we put together what seems like a reasonable model and then roll the dice. The danger lies, of course, in what happens after a researcher "rolls the dice" - do they present the results more-or-less as found, or do they pick them up and roll again? The latter, as has been shown repeatedly to the point that an entirely new vocabularly had to be propounded (the "False Positive" crisis, Non-replication crisis, etc.) to be more common in practice than any of us should find acceptable.

On the other hand, the group was quite *unlucky* as well. Working with the best technology and methods available at the time, informed by current best practices, and working under any number of early misconceptions at the level of theory and knowledge, they made judgment call after judgment call, as all researchers do. A reasonably strong effort was made to document these calls, though, in keeping with how things were done more than a decade ago, no serious effort was ever put forth to *systematically* document those seemingly trivial decisions, not to subject the analysis to the kind of rigorous documentation that is increasingly becoming the norm for some disciplines, journals, and research communities. So, through what truly seems like good fortune, there was an interesting artifact found. It was picked up, turned around a few times, and shared with a small group of others via a conference session. 

What may be the most remarkable part of this story is that it was nearly 10 years later, and another 2 of difficult analysis, before the original finding was laid to rest. But it __was laid to rest__. No retraction was necessary, but the final impact was the same. Through this careful re-analysis, we at least ensured that the original paper was never dusted off and submitted for publication in something approaching its original form.

I, for one, learned more than a dozen life-long lessons about statistical and analytical best practices - technical, procedural, philosophical, and even pedagogical - in a way that hammered these lessons deeply into my consciousness, not soon to be forgotten. While it did not restore to me my lost two years, I was able to use those same lessons to avoid costly mistakes in my other lines of research.

In fact, I'm not afraid to say that the whole affair left me rather unfit for the conventional research track, simply because I was now primed to see flimsy research results for exactly what they were, and to take proactive steps to suss out whether my other results would withstand the same sort of analysis. Spoiler: they often did not.

Effects that are substantial, robust, and relevant are hard to find. Artifacts and chimeras resulting from tortuous overfitting and other dubious practices are, unfortunately about as common as Iron Pyrite. 

Anyway, here's one of 10 do-files to produce those final 200 models:

In [ ]:
/*************************************************************************************/
/* This File is to be used in generating the large sensitivity-testing table project */
/* Started on 4 January 2006 There will be many of these that look very similar!!!!! */
/*************************************************************************************/


log close
log using paagrd01, replace text

set mem 30m

/***************************************************************************************************/
/* Regression using variable group "A1"                                                             */
/* Samples are, respectively: 1996, match (96 selection), match (05 selection),                    */   
/* 2005 not married, 2005 never married (when permissible)                                         */
/***************************************************************************************************/

/* AGES 3-12 */

use m03_12_2.dta  /* 1996 SAMPLE - 16233 cases */

/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/* AGES 13-22 */

use m13_22_2.dta


/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/***************************************************************************************************/
/* Regression using variable group "A1"                                                             */
/* Samples are, respectively: 1996, match (96 selection), match (05 selection),                    */   
/* 2005 not married, 2005 never married (when permissible)                                         */
/***************************************************************************************************/

/* AGES 3-12 */

use m03_12_3.dta   /* 1996-2005 Match SAMPLE (1996 variable selection) - 16173 cases */

/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/* AGES 13-22 */

use m13_22_3.dta


/*1973*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit where94 sex84 age84 pcmig84 elct84 person84 ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



/***************************************************************************************************/
/* Regression using variable group "A1"                                                             */
/* Samples are, respectively: 1996, match (96 selection), match (05 selection),                    */   
/* 2005 not married, 2005 never married (when permissible)                                         */
/***************************************************************************************************/

/* AGES 3-12 */

use m03_12_5.dta   /* 1996-2005 Match SAMPLE (2005 variables selection) - 15961 cases */

/*1973*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/* AGES 13-22 */

use m13_22_5.dta


/*1973*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



/***************************************************************************************************/
/* Regression using variable group "A1"                                                             */
/* Samples are, respectively: 1996, match (96 selection), match (05 selection),                    */   
/* 2005 not married, 2005 never married (when permissible)                                         */
/***************************************************************************************************/

/* AGES 3-12 */

use m03_12_1.dta   /* 2005 NOT MARRIED SAMPLE - 16016 cases (-98 missing data) */

/*1973*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/* AGES 13-22 */

use m13_22_1.dta


/*1973*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 



/***************************************************************************************************/
/* Regression using variable group "A1"                                                             */
/* Samples are, respectively: 1996, match (96 selection), match (05 selection),                    */   
/* 2005 not married, 2005 never married (when permissible)                                         */
/***************************************************************************************************/

/* AGES 3-12 */

use m03_12_4.dta   /* 2005 NEVER MARRIED SAMPLE - 15978 cases (-98 missing data) */

/*1973*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 

           
/*1976*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 


/* AGES 13-22 */

use m13_22_4.dta


/*1973*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp73b3 gfor73b3, basecategory(1) robust cluster (vill84) 
 
/*1976*/

mlogit migstat male age pctmigs electric totpop ginicult hdjute84 gcob3 gnp76b3 gfor76b3, basecategory(1) robust cluster (vill84) 




As you can see, by now we had all learned to speak an entirely unique dialect that made sense only within the confines of this single research project (one of several even for me at that point). 

It had become necessary to develop these short-hands just to keep all of the moving pieces clear and separate.

From this point, we did enter a final phase of the project. This phase was marked by a major shift from my plate to the plates of all the spatial programmers on the team (we added a few, if I recall, at this point).

Through such techniques as pixel-by-pixel differencing and matching different land use and/or land cover class systems, they were able to produce a set of visuals that were both illuminating and frustrating. They essentially showed that in nearly every way that mattered, the underyling classifications and the summary measures that were built from those could vary wildly based upon how they were performed. Given the nature of the end to which we were trying to put those summary measures (a multi-level, multi-method, multi-disciplinary collaboration), it was unrealistic to expect any strong case to be made from the existing literature about which flavor among hundreds was the "right" approach. And, it was unfair of us to expect of our geographer colleagues that they somehow resolve this dilemma when we were all equally out of our element to the same extent when considering that the model and theory belonged to no one discipline. 

It was a tough pill to swallow, but after several long meeting spent staring at beautiful slides depicting before and after changes in the classifications of a single image as equal parts blue (change from forest to non-forest), yellow (change from non-forest to forest), and green (forest remaining as forest), we were left with but two unnapealing choices:
+ throw in the towel
+ try to salvage some portion of the work that went into trying to save the paper as an entirely different analysis - likely a geography paper on some aspect of what we learned about the classification process

The social scientists grimly voted for option A. The geographers, surprisingly perhaps, mostly voted for option A as well, perhaps feeling that the paper would not be a hugely significant contribution even under the best of circumstances. 

We adjourned to the restaurant bar downstairs.

Fin. 

### References

Goodman, S. N., Fanelli, D., & Ioannidis, J. P. (2016). What does research reproducibility mean?. *Science translational medicine*, 8(341), 341ps12-341ps12.

Plesser H. E. (2018). Reproducibility vs. Replicability: A Brief History of a Confused Terminology. *Frontiers in neuroinformatics*, 11, 76. https://doi.org/10.3389/fninf.2017.00076

Rindfuss, Ronald, Entwisle, Barbara, and Walsh, Steve. Nang Rong Projects [Thailand]. *Inter-university Consortium for Political and Social Research* [distributor], 2009-03-06. https://doi.org/10.3886/ICPSR04402.v5